<a href="https://colab.research.google.com/github/ZahraShourmeij/Spark/blob/master/runtime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sparknlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.5/537.5 kB 5.3 MB/s eta 0:00:00


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=0cc0d314d43b88c16ddc332218d08cab3821de0480efc192cafc0353fc92f480
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import sparknlp
from pyspark.sql import SparkSession
from pyspark import SparkContext
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover, NGram, CountVectorizer, HashingTF, IDF
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, SQLTransformer, VectorIndexer
#from sparknlp.annotator import Normalizer, Stemmer, Lemmatizer
#from sparknlp.base import DocumentAssembler
from pyspark.sql import functions as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections
from wordcloud import WordCloud
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import when, col
from pyspark.ml.clustering import KMeans
from pyspark import SparkContext
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.types import StringType
from pyspark.storagelevel import StorageLevel
import time
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, NaiveBayes
from pyspark import SparkConf

import time

try:
    # Start measuring the time
    start_time = time.time()

    spark = sparknlp.start()

    cluster_url = "spark://ms0836.utah.cloudlab.us:7077"

    # Create a SparkConf object
    spark_conf = SparkConf()
    spark_conf.set("spark.executorEnv.LD_LIBRARY_PATH", "/usr/lib/x86_64-linux-gnu/libopenblas.so")
    spark_conf.set("spark.driverEnv.LD_LIBRARY_PATH", "/usr/lib/x86_64-linux-gnu/libopenblas.so")
    #spark_conf.set("spark.some_config_option", "config_value")
    spark_conf.set("spark.executor.cores", "4")


    # Create a Spark session
    spark = SparkSession.builder \
    .appName("MySparkApp") \
    .config("spark.driver.memory", "32g") \
    .config("spark.executor.memory", "32g") \
    .config("spark.driver.memoryOverhead", "2g") \
    .config("spark.executor.instances", "3") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "32g") \
    .config("spark.broadcast.blockSize", "64m") \
    .config("spark.sql.autoBroadcastJoinThreshold", 100 * 1024 * 1024) \
    .config(conf=spark_conf) \
    .master(cluster_url) \
    .getOrCreate()


    sc = SparkContext.getOrCreate()

    #content = "1. T1-T2 DATE**[12/24/13] $1.99 () (10/12), ph+ 90%"
    pattern = "\s+|(?=[-.:;*+,$&%\\[\\]])|(?<=[-.:;*+, &%\[\]])|\\b\\w+\\b"


    #train_data = spark.read.csv("EXIST2021_training16.csv", header=True, inferSchema=True)

    train_data = spark.read.option("header","true") \
                        .option("sep", "\t") \
                        .option("multiLine", "true") \
                        .option("quote","\"") \
                        .option("escape","\"") \
                        .option("ignoreTrailingWhiteSpace", True) \
                        .csv("EXIST2021_training.tsv")


    train_data = train_data.filter(col("language") == "en")

    train_data = train_data.filter(~col("task1").isNull())
    #You can also use the na.drop() method to remove rows with null values
    #val dfWithoutNull = df.na.drop()
    # Filter out rows with null values in the 'text' column
    train_data = train_data.filter(col("text").isNotNull())

    # Repartition the DataFrame into 8 partitions
    train_data = train_data.repartition(8)

    # Coalesce the DataFrame into 4 partitions
    train_data = train_data.coalesce(4)

    # Convert your DataFrame to an RDD
    rdd = train_data.rdd

    # Cache a DataFrame in memory
    train_data.persist()

    # You can also specify the storage level (memory and/or disk)
    train_data.persist(StorageLevel.MEMORY_AND_DISK)

    train_data.show()


# Transforms raw texts to `document` annotation
    documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")

    sentenceDetector = SentenceDetector()\
        .setInputCols(['document'])\
        .setOutputCol('sentences')\


    regexTokenizer = RegexTokenizer() \
        .setInputCols(["sentences"]) \
        .setOutputCol("regexToken") \
        .setPattern(pattern) \
        .setPositionalMask(False)

    #Default is "<[^>]*>" (e.g., it removes all HTML tags).
    #normalizer = Normalizer().setInputCols(["regexToken"]).setOutputCol("normalized")
    # remove punctuations (keep alphanumeric chars)
        # if we don't set CleanupPatterns, it will only keep alphabet letters ([^A-Za-z])
    #\w :It includes uppercase and lowercase letters from A to Z, digits from 0 to 9, and the underscore character ().
    #\d : digits
    #\s : whitespace
    normalizer = Normalizer() \
        .setInputCols("regexToken") \
        .setOutputCol("normalized") \
        .setLowercase(True) \
    #.setCleanupPatterns(["[^\w\d\s]"])

    #stopwords = StopWordsCleaner().getStopWords() + ["word1", "word2", "word3"]
    stopwordList = ["now"]
    stopwordList.extend(StopWordsRemover().getStopWords())
    stopwordList = list(set(stopwordList))#optionnal

    #stopWordsRemover = StopWordsRemover().setInputCols(["normalized"]).setOutputCol("cleaned")
    stopwords_cleaner = StopWordsCleaner()\
        .setInputCols("normalized")\
        .setOutputCol("cleanTokens")\
        .setCaseSensitive(False)\
        .setStopWords(stopwordList)

    tokenassembler = TokenAssembler()\
        .setInputCols(["sentences", "cleanTokens"]) \
        .setOutputCol("clean_text")

    #stemmer = Stemmer().setInputCols(["clean_text"]).setOutputCol("stemmed")
    stemmer = Stemmer() \
        .setInputCols(["cleanTokens"]) \
        .setOutputCol("stemmed")

    #lemmatizer = Lemmatizer().setInputCols(["stemmed"]).setOutputCol("lemmatized")
    lemmatizer = Lemmatizer() \
        .setInputCols(["cleanTokens"]) \
        .setOutputCol("lemma") \
        .setDictionary("./AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")

    #ngram = NGram().setN(2).setInputCols(["lemmatized"]).setOutputCol("ngrams")
    ngrams_cum = NGramGenerator() \
                .setInputCols(["lemma"]) \
                .setOutputCol("ngrams") \
                .setN(3) \
                .setEnableCumulative(True)\
                .setDelimiter("_") # Default is space


    finisher = Finisher() \
        .setInputCols(["ngrams"]) \
        .setIncludeMetadata(False) # set to False to remove metadata\

    # TF-IDF vectorization
    hashingTF = HashingTF(inputCol="finished_ngrams", outputCol="rawFeatures")
    idf = IDF(inputCol="rawFeatures", outputCol="features")


    # we will remove the words that appear in 8 docs or less.
    #count_vector = CountVectorizer(minTF=1, minDF=8, vocabSize=2**17).setInputCol("finished_ngrams").setOutputCol("TF")

    countVectors = CountVectorizer(inputCol="finished_ngrams", outputCol="featuresCount", vocabSize=10000, minDF=5)

    # Set maxCategories so features with > 4 distinct values are treated as continuous.
    featureIndexer =\
        VectorIndexer(inputCol="featuresCount", outputCol="indexedFeatures", maxCategories=4)

    label_stringIdx = StringIndexer(inputCol = "task1", outputCol = "label", handleInvalid="keep")


    # Create pipeline
    pipeline = Pipeline(
        stages=[
            documentAssembler,
            sentenceDetector,
            regexTokenizer,
            normalizer,
            stopwords_cleaner,
            tokenassembler,
            stemmer,
            lemmatizer,
            ngrams_cum,
            finisher,
            hashingTF,
            idf,
            countVectors,
            featureIndexer,
            label_stringIdx
            ])

    # Fit the pipeline to the dataset
    pipelineModel = pipeline.fit(train_data)

    # Transform the dataset
    processedTrainSet = pipelineModel.transform(train_data)

    # Show the processed dataset
    processedTrainSet.show(truncate=False)

    processedTrainSet.select('text','finished_ngrams').show(truncate=50)

    #processedTrainSet = processedTrainSet.withColumn("labelInt", col("label").cast("int"))

    processedTrainSet.select('text','task1','label','features','indexedFeatures').show(truncate=50)



    test_data = spark.read.option("header","true") \
                    .option("sep", "\t") \
                    .option("multiLine", "true") \
                    .option("quote","\"") \
                    .option("escape","\"") \
                    .option("ignoreTrailingWhiteSpace", True) \
                    .csv("EXIST2021_test_labeled.tsv")


    test_data = test_data.filter(col("language") == "en")

    test_data = test_data.filter(~col("task1").isNull())
    # Filter out rows with null values in the 'text' column
    test_data = test_data.filter(col("text").isNotNull())

    # Repartition the DataFrame into 8 partitions
    test_data = test_data.repartition(8)

    # Coalesce the DataFrame into 4 partitions
    test_data = test_data.coalesce(4)

    # Convert your DataFrame to an RDD
    rdd = test_data.rdd

    # Cache a DataFrame in memory
    test_data.persist()

    # You can also specify the storage level (memory and/or disk)
    test_data.persist(StorageLevel.MEMORY_AND_DISK)

    test_data.show()


    # Transforms raw texts to `document` annotation
    documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")

    sentenceDetector = SentenceDetector()\
        .setInputCols(['document'])\
        .setOutputCol('sentences')\


    regexTokenizer = RegexTokenizer() \
        .setInputCols(["sentences"]) \
        .setOutputCol("regexToken") \
        .setPattern(pattern) \
        .setPositionalMask(False)

    #Default is "<[^>]*>" (e.g., it removes all HTML tags).
    #normalizer = Normalizer().setInputCols(["regexToken"]).setOutputCol("normalized")
    # remove punctuations (keep alphanumeric chars)
        # if we don't set CleanupPatterns, it will only keep alphabet letters ([^A-Za-z])
    #\w :It includes uppercase and lowercase letters from A to Z, digits from 0 to 9, and the underscore character ().
    #\d : digits
    #\s : whitespace
    normalizer = Normalizer() \
        .setInputCols("regexToken") \
        .setOutputCol("normalized") \
        .setLowercase(True) \
        #.setCleanupPatterns(["[^\w\d\s]"])

    #stopwords = StopWordsCleaner().getStopWords() + ["word1", "word2", "word3"]
    stopwordList = ["now"]
    stopwordList.extend(StopWordsRemover().getStopWords())
    stopwordList = list(set(stopwordList))#optionnal

    #stopWordsRemover = StopWordsRemover().setInputCols(["normalized"]).setOutputCol("cleaned")
    stopwords_cleaner = StopWordsCleaner()\
        .setInputCols("normalized")\
        .setOutputCol("cleanTokens")\
        .setCaseSensitive(False)\
        .setStopWords(stopwordList)

    tokenassembler = TokenAssembler()\
        .setInputCols(["sentences", "cleanTokens"]) \
        .setOutputCol("clean_text")

    #stemmer = Stemmer().setInputCols(["clean_text"]).setOutputCol("stemmed")
    stemmer = Stemmer() \
        .setInputCols(["cleanTokens"]) \
        .setOutputCol("stemmed")

    #lemmatizer = Lemmatizer().setInputCols(["stemmed"]).setOutputCol("lemmatized")
    lemmatizer = Lemmatizer() \
        .setInputCols(["cleanTokens"]) \
        .setOutputCol("lemma") \
        .setDictionary("./AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")

    #ngram = NGram().setN(2).setInputCols(["lemmatized"]).setOutputCol("ngrams")
    ngrams_cum = NGramGenerator() \
                    .setInputCols(["lemma"]) \
                    .setOutputCol("ngrams") \
                    .setN(3) \
                    .setEnableCumulative(True)\
                    .setDelimiter("_") # Default is space


    finisher = Finisher() \
        .setInputCols(["ngrams"]) \
        .setIncludeMetadata(False) # set to False to remove metadata\

    # TF-IDF vectorization
    hashingTF = HashingTF(inputCol="finished_ngrams", outputCol="rawFeatures")
    idf = IDF(inputCol="rawFeatures", outputCol="features")


    # we will remove the words that appear in 8 docs or less.
    #count_vector = CountVectorizer(minTF=1, minDF=8, vocabSize=2**17).setInputCol("finished_ngrams").setOutputCol("TF")

    countVectors = CountVectorizer(inputCol="finished_ngrams", outputCol="featuresCount", vocabSize=10000, minDF=5)

    # Set maxCategories so features with > 4 distinct values are treated as continuous.
    featureIndexer =\
        VectorIndexer(inputCol="featuresCount", outputCol="indexedFeatures", maxCategories=4)

    label_stringIdx = StringIndexer(inputCol = "task1", outputCol = "label", handleInvalid="keep")



    # Create pipeline
    pipeline = Pipeline(
        stages=[
            documentAssembler,
            sentenceDetector,
            regexTokenizer,
            normalizer,
            stopwords_cleaner,
            tokenassembler,
            stemmer,
            lemmatizer,
            ngrams_cum,
            finisher,
            hashingTF,
            idf,
            countVectors,
            featureIndexer,
            label_stringIdx
            ])

    # Fit the pipeline to the dataset
    pipelineModel = pipeline.fit(test_data)

    # Transform the dataset
    processedTestSet = pipelineModel.transform(test_data)

    # Show the processed dataset
    processedTestSet.show(truncate=False)

    processedTestSet.select('text','finished_ngrams').show(truncate=50)

    #processedTrainSet = processedTrainSet.withColumn("labelInt", col("label").cast("int"))

    processedTrainSet.select('text','task1','label','features','indexedFeatures').show(truncate=50)


    print("Training Dataset Count: " + str(processedTrainSet.count()))
    print("Test Dataset Count: " + str(processedTestSet.count()))

    from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
    from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier

    rf = RandomForestClassifier(labelCol="label", featuresCol="features")

    rfModel = rf.fit(processedTrainSet)

    random_forest_prediction = rfModel.transform(processedTestSet)

    # Select example rows to display.
    #predictions.select("prediction", "features").show(5)

    # Join the train_data and predictions based on a common identifier (e.g., "id")
    #joined_data = processedTrainSet.join(predictions, on="id", how="inner")

    #how many of the predicted positive instances were actually positive.
    evaluator = MulticlassClassificationEvaluator(
        labelCol="label", predictionCol="prediction", metricName="weightedPrecision")

    precision = evaluator.evaluate(random_forest_prediction)
    print(f"Weighted Precision: {precision}")

    # Calculate accuracy
    accuracy = evaluator.evaluate(random_forest_prediction, {evaluator.metricName: "accuracy"})
    print(f"Accuracy: {accuracy}")

    # Calculate F1-score
    f1 = evaluator.evaluate(random_forest_prediction, {evaluator.metricName: "f1"})
    print(f"F1-Score: {f1}")

    # Calculate recall : how many of the actual positive instances were correctly predicted as positive.
    recall = evaluator.evaluate(random_forest_prediction, {evaluator.metricName: "weightedRecall"})
    print(f"Weighted Recall: {recall}")
    #rfModel = model.stages[2]
    #print(rfModel)  # summary only

    lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, labelCol="label")

    # Fit the model
    lrModel = lr.fit(processedTrainSet)

    # Print the coefficients and intercept for multinomial logistic regression
    #These values define the decision boundary of the model.
    print("Coefficients: \n" + str(lrModel.coefficientMatrix))
    print("Intercept: " + str(lrModel.interceptVector))

    trainingSummary = lrModel.summary

    accuracy = trainingSummary.accuracy
    falsePositiveRate = trainingSummary.weightedFalsePositiveRate
    truePositiveRate = trainingSummary.weightedTruePositiveRate
    fMeasure = trainingSummary.weightedFMeasure()
    precision = trainingSummary.weightedPrecision
    recall = trainingSummary.weightedRecall
    print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
          % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

    # Train a DecisionTree model.
    dt = DecisionTreeClassifier(labelCol="label", featuresCol="indexedFeatures")

    # Chain indexers and tree in a Pipeline
    #pipeline = Pipeline(stages=[dt])

    # Train model.  This also runs the indexers.
    model = dt.fit(processedTrainSet)

    # Make predictions.
    decision_tree_prediction = model.transform(processedTestSet)

    # Select example rows to display.
    #predictions.select("prediction", "label", "features").show(5)

    # Select (prediction, true label) and compute test error
    evaluator = MulticlassClassificationEvaluator(
        labelCol="label", predictionCol="prediction", metricName="accuracy")
    accuracy = evaluator.evaluate(decision_tree_prediction)
    print("Test Error = %g " % (1.0 - accuracy))


    from pyspark.ml.classification import NaiveBayes
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator

    # create the trainer and set its parameters
    #Smoothing parameter (Laplace smoothing) to handle unseen features during prediction.
    nb = NaiveBayes(smoothing=1.0, modelType="multinomial", labelCol="label")

    # train the model
    model = nb.fit(processedTrainSet)

    # select example rows to display.
    naive_bayes_predictions = model.transform(processedTestSet)
    #predictions.show()

    # compute accuracy on the test set
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                                  metricName="accuracy")
    accuracy = evaluator.evaluate(naive_bayes_predictions)
    print("Test set accuracy = " + str(accuracy))






finally:
    # Stop the Spark session
    spark.stop()
    sc.stop()

    # Calculate the runtime
    end_time = time.time()
    runtime = end_time - start_time

    print(f"Total runtime: {runtime} seconds")


+---------+----+-------+--------+--------------------+----------+--------------------+
|test_case|  id| source|language|                text|     task1|               task2|
+---------+----+-------+--------+--------------------+----------+--------------------+
|EXIST2021|1019|twitter|      en|@MeechiiMeech To ...|non-sexist|          non-sexist|
|EXIST2021|2563|twitter|      en|Please I’m beggin...|    sexist|     sexual-violence|
|EXIST2021|2236|twitter|      en|The Slovenian Slu...|non-sexist|          non-sexist|
|EXIST2021| 110|twitter|      en|You know the worl...|non-sexist|          non-sexist|
|EXIST2021|3182|twitter|      en|It’s hard finding...|    sexist|misogyny-non-sexu...|
|EXIST2021| 541|twitter|      en|@thedollmari @tra...|non-sexist|          non-sexist|
|EXIST2021|3415|twitter|      en|Little Mix: Wings...|non-sexist|          non-sexist|
|EXIST2021|3208|twitter|      en|My bedtime read b...|non-sexist|          non-sexist|
|EXIST2021|2169|twitter|      en|Some Women